## Restaurant ratings prediction

In [2]:
from IPython.display import display
from sklearn.model_selection import train_test_split
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL of the Yelp search results page
url = "https://fr.yelp.be/search?find_desc=&find_loc=Brussels%2C+R%C3%A9gion+de+Bruxelles-Capitale"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all the restaurant cards on the page
restaurant_cards = soup.find_all("div", class_="lemon--div__373c0__1mboc container__373c0__3HMKB businessName__373c0__1fTgn")

# Initialize lists to store the extracted data
attributes_list = []
ratings_list = []

# Iterate over each restaurant card
for card in restaurant_cards:
    # Extract the restaurant attributes
    attributes = []
    for attribute_element in card.find_all("div", class_="lemon--div__373c0__1mboc arrange-unit__373c0__o3tjT arrange-unit-fill__373c0__1bXMl border-color--default__373c0__3-ifU"):
        attributes.append(attribute_element.text.strip())
    
    # Extract the restaurant rating
    rating_element = card.find("div", class_="lemon--div__373c0__1mboc i-stars__373c0__1T6rz i-stars--regular-4-half__373c0__3oMI2 border-color--default__373c0__3-ifU overflow--hidden__373c0__2y4YK")
    rating = float(rating_element["aria-label"].split()[0]) if rating_element else None
    
    # Append the extracted data to the lists
    attributes_list.append(attributes)
    ratings_list.append(rating)

# Create a pandas DataFrame to store the scraped data
data = pd.DataFrame({
    "Attributes": attributes_list,
    "Rating": ratings_list
})

# Print the DataFrame
print(data)


Empty DataFrame
Columns: [Attributes, Rating]
Index: []


In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# Set up Chrome WebDriver
s = Service('path_to_chromedriver')
options = Options()
options.add_argument('--headless')  # Run Chrome WebDriver in headless mode
driver = webdriver.Chrome(service=s, options=options)

# Define the URL of the Yelp search results page
url = "https://fr.yelp.be/search?find_desc=&find_loc=Brussels%2C+R%C3%A9gion+de+Bruxelles-Capitale"

# Navigate to the Yelp search results page
driver.get(url)

# Wait for the page to load
time.sleep(5)

# Scroll down the page to load more restaurants (optional)
scroll_element = driver.find_element(By.XPATH, "//body")
for _ in range(3):  # Scroll down 3 times, adjust as needed
    ActionChains(driver).move_to_element(scroll_element).send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(2)  # Pause to load more restaurants

# Find all the restaurant cards on the page
restaurant_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'container__373c0__3HMKB')]")

# Initialize lists to store the extracted data
attributes_list = []
ratings_list = []

# Iterate over each restaurant card
for card in restaurant_cards:
    # Extract the restaurant attributes
    attributes = card.find_elements(By.XPATH, ".//div[contains(@class, 'border-color--default__373c0__3-ifU')]")
    attributes = [attr.text.strip() for attr in attributes]
    
    # Extract the restaurant rating
    rating_element = card.find_element(By.XPATH, ".//div[contains(@class, 'i-stars__373c0__1T6rz')]")
    rating = float(rating_element.get_attribute("aria-label").split()[0]) if rating_element else None
    
    # Append the extracted data to the lists
    attributes_list.append(attributes)
    ratings_list.append(rating)

# Create a pandas DataFrame to store the scraped data
data = pd.DataFrame({
    "Attributes": attributes_list,
    "Rating": ratings_list
})

# Print the DataFrame
print(data)

# Quit the WebDriver
driver.quit()


Empty DataFrame
Columns: [Attributes, Rating]
Index: []


In [9]:
!pip install yelpapi

from yelpapi import YelpAPI
import pandas as pd

# Initialize Yelp API client with your API key
api_key = "YOUR_API_KEY"
yelp_api = YelpAPI(api_key)

# Set the search parameters
location = "Brussels, Belgium"

# Make the API call to search for restaurants
response = yelp_api.search_query(location=location)

# Extract the relevant data from the response
restaurants = response["businesses"]
data = []
for restaurant in restaurants:
    attributes = restaurant["attributes"]
    rating = restaurant["rating"]
    data.append({
        "Attributes": attributes,
        "Rating": rating
    })

# Create a pandas DataFrame to store the scraped data
data_df = pd.DataFrame(data)

# Print the DataFrame
print(data_df)


YelpAPIError: VALIDATION_ERROR: 'Bearer YOUR_API_KEY' does not match '^(?i)Bearer [A-Za-z0-9\\-\\_]{128}$'

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into features (attributes) and target (rating)
X = data["Attributes"]
y = data["Rating"]

# Convert the attributes into a binary matrix using one-hot encoding
X_encoded = pd.get_dummies(X.apply(pd.Series).stack()).sum(level=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs")
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


AttributeError: 'Series' object has no attribute 'stack'